## How to use this code

#### To see each experiment in EEG
eeg_process = EEGProcessor(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv", time_interval = 20, remove_time_in_group = 15) <br>
#### To get total experiment for each person
eeg_process = EEGProcessor(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv", time_interval = 20, remove_time_in_group = 15) <br>
data = pd.read_csv(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv") <br>
위의 해당 파트에서 경로 변경, time_interval, remove_time_in_group 변경하면서 사용하면 됨. <br>
time_interval : 그룹화 할 초 단위 (ex. 20초)<br>
remove_time_in_group : 그룹 내에서 오류값 처리의 기준이 되는 초 (ex. 15초). 예를 들어, 15초 이상의 값이 존재하지만 20초까지는 안 되는 그룹의 경우 (ex. 16초) 는 이 값을 살려 해당 구간을 임의로 늘려서 16초간의 평균값을 20초 구간의 대표값으로 가져가도록 한다. 


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

class EEGProcessor:
    #time_interval : Fitbit과 통일할 초 (ex. 20초) 
    #remove_time_in_group : time_interval 그룹 내에서 오류값 처리의 threshold로 사용할 기준 초
    def __init__(self, file_path, time_interval, remove_time_in_group):
        self.time_interval = time_interval
        self.remove_time_in_group = remove_time_in_group
        self.time_interval_str = f'{time_interval}S'
        self.EEG_report = pd.read_csv(file_path)

    #리스트 값을 row로 변환하는 함수
    def parse_raw_data(self, dataframe, col_name):
        col_str = dataframe.iloc[0][col_name]
        col_str = col_str.strip('[]')
        col_list = [float(val) for val in col_str.split(',')]  # 쉼표로 구분된 값을 리스트로 변환
        col_data = pd.DataFrame({col_name: col_list})
        return col_data

    #측정 시간 출력 함수
    def time_difference(self, dataframe, start_time_col, finish_time_col):
        start_time = datetime.strptime(dataframe.iloc[0][start_time_col], '%Y-%m-%d %H:%M:%S')
        finish_time = datetime.strptime(dataframe.iloc[0][finish_time_col], '%Y-%m-%d %H:%M:%S')

        # 두 datetime 객체 간의 차이 계산
        time_difference = (finish_time - start_time).total_seconds()
        return time_difference
    
    #실험 초기 인식 오류 기간 비교 후, 삭제할 부분 삭제
    def count_initial_same_values(self, series):
        initial_value = series.iloc[0]
        count = 0
        for value in series:
            if value == initial_value:
                count += 1
            else:
                break
        return count
    
    # 앞, 뒤로 0,20,40초 단위로 잘리지 않는 값들에 대한 처리 (15초 이상이면 살리기)
    def process_start_time_trash_sec(self, dt):
        # 입력된 시간에서 분은 1 더하고 초는 버림
        rounded_time = dt + timedelta(minutes=1) - timedelta(seconds=dt.second)
        time_difference = (rounded_time - dt).total_seconds()

        # 20초로 나눈 나머지 계산 ex. 12
        remainder = time_difference % float(self.time_interval)

        # 나머지가 15초 미만의 데이터는 버리고, 15초 이상의 데이터는 버리는 것 없이 그대로 사용하기
        if self.remove_time_in_group <= remainder:
            return False
        else:
        # ex. 2023-10-15 00:21:04 -> remainder : 16 (False)
        # ex. 2023-10-15 00:21:15 -> remainder : 5
            return remainder

    def process_finish_time_trash_sec(self, dt):
        # 입력된 시간에서 초는 버림
        rounded_time = dt - timedelta(seconds=dt.second)
        time_difference = (dt - rounded_time).total_seconds()

        # 20초로 나눈 나머지 계산
        remainder = time_difference % float(self.time_interval)

        if self.remove_time_in_group <= remainder:
            return False

        else:
        #ex. 2023-10-15 00:40:46 -> remainder : 6
        #ex. 2023-10-15 00:40:56 -> remainder : 16 (False)
            return remainder

    def nearest_time_rounding(self, dt):
        # 주어진 시간의 초를 추출
        seconds = dt.second
        # 0, 20, 40초 중 가장 가까운 값을 찾기 위한 리스트
        time_points = [i for i in range(0,60, self.time_interval)]
        # 가장 가까운 초 값 찾기
        nearest = min(time_points, key=lambda x: abs(x - seconds))
        # 찾은 초 값으로 시간 변경
        # 만약 nearest가 40이고 seconds가 55보다 크다면, 분을 1 더해주고 초를 0으로 설정
        if nearest == time_points[-1] and seconds >= (time_points[-1] + self.remove_time_in_group):
            rounded_time = dt.replace(second=0, microsecond=0) + timedelta(minutes=1)
        else:
            rounded_time = dt.replace(second=nearest, microsecond=0)

        return rounded_time

    # 실험 종료 시간 맞춰주기
    # 더 늦게 끝나는 데이터프레임을 일찍 끝나는 데이터프레임에 맞춰야 함
    def align_end_time(self, df1, df2):
        if df1.index[-1] > df2.index[-1]:
            df1 = df1[df1.index <= df2.index[-1]]

        elif df1.index[-1] < df2.index[-1]:
            df2 = df2[df2.index <= df1.index[-1]]

        else: 
            pass # 두 데이터프레임의 종료 시간이 동일한 경우

        return df1, df2

    # start time 가공 -> i : 0 , finish time 가공 -> i : -1
    # start time 가공 -> process_start_time_trash_sec 함수 , finish time 가공 -> process_finish_time_trash_sec 함수
    def adjust_time_index(self, i, df, func):
        remainder = func(df.index[i])
        #finish 부분에서 00초, 20초, 40초로 끝나면 가공된 마지막 row는 한 개의 데이터로 이루어지기 때문에 1초를 빼서 59, 19, 39초에서 마무리하고자
        one_sec = timedelta(seconds=1)

        if remainder == False:
            # time을 가장 가까운 0, 20, 40초 중 하나로 변경
            if i == 0 :
                time = self.nearest_time_rounding(df.index[i])
                new_index = df.index.tolist()
                new_index[i] = time
                df.index = new_index
            else:
                time = self.nearest_time_rounding(df.index[i]) - one_sec
                new_index = df.index.tolist()
                new_index[i] = time
                df.index = new_index

        else:
            cutting_time = timedelta(seconds=remainder)        
            # remainder가 15초 미만이면 버리고 그 이후부터 시작
            if i == 0:
                df = df[df.index >= df.index[i] + cutting_time]
            # remainder가 15초 미만이면 버리고 그 이전 0,20,40초 중으로 마무리
            else:
                df = df[df.index <= df.index[-1] - cutting_time - one_sec]

        return df
    
    def check_invalid_values(self, group):
        # brain wave에서 연속 0 차이값을 가지는 시간 구간 계산
        alpha_invalid_series = group['α_wave_raw_data'].diff().eq(0)
        alpha_invalid_timestamps = group.index[alpha_invalid_series].tolist()

        # attention_raw_data에서 0 값을 가지는 시간 구간 계산
        attention_invalid_series = group['attention_raw_data'] == 0
        attention_invalid_timestamps = group.index[attention_invalid_series].tolist()

        # 연속 0 차이값 또는 0 값을 가지는 시간 구간의 길이가 15초 이상인지 확인
        def has_long_invalid_duration(invalid_timestamps):
            if not invalid_timestamps:
                return False
            for i in range(1, len(invalid_timestamps)):
                if (invalid_timestamps[i] - invalid_timestamps[i-1]).seconds > self.remove_time_in_group:
                    return True
            return False

        alpha_invalid = has_long_invalid_duration(alpha_invalid_timestamps)
        attention_invalid = has_long_invalid_duration(attention_invalid_timestamps)

        if alpha_invalid or attention_invalid:
            return pd.Series([np.nan] * group.shape[1], index=group.columns)

        else:
            # 오류 값을 제외하고 평균 계산
            valid_conditions = (
                (group['α_wave_raw_data'].diff() != 0) & 
                (group['β_wave_raw_data'].diff() != 0) & 
                (group['θ_wave_raw_data'].diff() != 0) & 
                (group['δ_wave_raw_data'].diff() != 0) & 
                (group['γ_wave_raw_data'].diff() != 0) & 
                (group['attention_raw_data'] != 0)
            )
            return group[valid_conditions].mean()
        
    def check_invalid_values_other(self, group):
        # hr에서 0 값을 가지는 시간 구간 계산
        hr_invalid_series = group['hr_raw_data'] == 0
        hr_invalid_timestamps = group.index[hr_invalid_series].tolist()

        # 연속 0 차이값 또는 0 값을 가지는 시간 구간의 길이가 15초 이상인지 확인
        def has_long_invalid_duration(invalid_timestamps):
            if not invalid_timestamps:
                return False
            for i in range(1, len(invalid_timestamps)):
                if (invalid_timestamps[i] - invalid_timestamps[i-1]).seconds > self.remove_time_in_group:
                    return True
            return False

        hr_invalid = has_long_invalid_duration(hr_invalid_timestamps)

        if hr_invalid:
            return pd.Series([np.nan] * group.shape[1], index=group.columns)

        else:
            # 오류 값을 제외하고 평균 계산
            group = group[(group['hr_raw_data'] != 0)]
            return group.mean()
    
    def process_data(self, experiment_id):
        if experiment_id not in self.EEG_report.index:
            return None

        # 모든 실험들 for문으로 돌리고 하나의 데이터프레임으로 저장되어야 함.
        EEG_report_sample = self.EEG_report.loc[[experiment_id],:]

        #한 column당 하나의 dataframe
        cols = ['α_wave_raw_data', 'β_wave_raw_data', 'θ_wave_raw_data', 'δ_wave_raw_data', 'γ_wave_raw_data', 'attention_raw_data', 'hrv_raw_data', 'hr_raw_data', 'coherence_flag_raw_data']
        parsed_dfs = [self.parse_raw_data(EEG_report_sample, col) for col in cols]

        #interval second 계산
        interval_sec = self.time_difference(EEG_report_sample, 'meditation_start_time', 'meditation_finish_time') / len(parsed_dfs[0])
        interval_sec_other = self.time_difference(EEG_report_sample, 'meditation_start_time', 'meditation_finish_time') / len(parsed_dfs[6])

        #병합된 dataframe 생성
        merged_df = parsed_dfs[0].join(parsed_dfs[1:6])
        merged_df_other = parsed_dfs[6].join(parsed_dfs[7:])

        #실험 시작 시간
        start_time = datetime.strptime(EEG_report_sample.iloc[0]['meditation_start_time'], '%Y-%m-%d %H:%M:%S')

        #실험 feature 별 interval sec 이용하여 실험 시간 time index로 데이터프레임 변환 (2가지)
        interval_sec, interval_sec_other = timedelta(seconds=round(interval_sec,2)), timedelta(seconds=round(interval_sec_other,2))
        merged_df['time'] = [start_time + i * interval_sec for i in range(len(merged_df))]
        merged_df_other['time'] = [start_time + i * interval_sec_other for i in range(len(merged_df_other))]
        merged_df, merged_df_other = merged_df.set_index('time'), merged_df_other.set_index('time')
        
        #실험 초기 인식 오류 기간 비교용
        counts = [self.count_initial_same_values(merged_df[col]) for col in cols[:6]] + [self.count_initial_same_values(merged_df_other['hr_raw_data'])]
        
        #interval_sec 단위 float로 변경하여 비교
        initial_error_times = [counts[i] * interval_sec.total_seconds() if i != 6 else counts[i] * interval_sec_other.total_seconds() for i in range(7)]
        
        # feature 별 초기 오류 시간 비교 후, 가장 오류 시간이 긴 것에 맞춰 initial error time 설정
        initial_error_time = timedelta(seconds=max(initial_error_times))

        # 초기 인식 오류 제거한 실험 data 시작 시간
        real_start_time = start_time + initial_error_time
        merged_df, merged_df_other = merged_df[merged_df.index > real_start_time], merged_df_other[merged_df_other.index > real_start_time]

        # 데이터프레임 인덱스를 초 단위로 반올림
        merged_df.index, merged_df_other.index = merged_df.index.round('S'), merged_df_other.index.round('S')

        # 실험 종료 시간 맞춰주기
        merged_df, merged_df_other = self.align_end_time(merged_df, merged_df_other)

#         # 더 늦게 끝나는 데이터프레임을 일찍 끝나는 데이터프레임에 맞춰야 함.
#         if merged_df.index[-1] > merged_df_other.index[-1]:
#             merged_df = merged_df[merged_df.index <= merged_df_other.index[-1]]

#         elif merged_df.index[-1] < merged_df_other.index[-1] :
#             merged_df_other = merged_df_other[merged_df_other.index <= merged_df.index[-1]]
            
#         else:
#             True

        # start time 가공 -> i : 0 , finish time 가공 -> i : -1
        # start time 가공 -> process_start_time_trash_sec 함수 , finish time 가공 -> process_finish_time_trash_sec 함수
        merged_df = self.adjust_time_index(0, merged_df, self.process_start_time_trash_sec)
        merged_df_other = self.adjust_time_index(0, merged_df_other, self.process_start_time_trash_sec)
        merged_df = self.adjust_time_index(-1, merged_df, self.process_finish_time_trash_sec)
        merged_df_other = self.adjust_time_index(-1, merged_df_other, self.process_finish_time_trash_sec)

        # 20초 단위로 그룹화 (인덱스가 datetime 형태이므로 floor 사용)
        grouped = merged_df.groupby(merged_df.index.floor(self.time_interval_str))
        grouped_other = merged_df_other.groupby(merged_df_other.index.floor(self.time_interval_str))

        result = grouped.apply(self.check_invalid_values)
        result_other = grouped_other.apply(self.check_invalid_values_other)

        # β/θ SP ratio를 포함한 최종 EEG 데이터셋
        EEG_data_per_time_interval = result.merge(result_other, left_index=True, right_index=True)
        EEG_data_per_time_interval['β/θ SP'] = EEG_data_per_time_interval['β_wave_raw_data'] / EEG_data_per_time_interval['θ_wave_raw_data']
        
        return EEG_data_per_time_interval

In [2]:
# eeg_process = EEGProcessor(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv", time_interval = 20, remove_time_in_group = 15)
# eeg_process.process_data(10)

In [3]:
# eeg_process_extended = EEGProcessorExtended(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv", time_interval=20, remove_time_in_group=15)
eeg_process = EEGProcessor(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv", time_interval = 20, remove_time_in_group = 16)
data = pd.read_csv(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm.csv")
result_dfs = []

for exp_id in range(3, len(data)):
    processed_data = eeg_process.process_data(exp_id)
    if processed_data is not None:
        result_dfs.append(processed_data)
        
if result_dfs:
    combined_df = pd.concat(result_dfs)
    
combined_df

,α_wave_raw_data,β_wave_raw_data,θ_wave_raw_data,δ_wave_raw_data,γ_wave_raw_data,attention_raw_data,hrv_raw_data,hr_raw_data,coherence_flag_raw_data,β/θ SP
2023-10-17 16:13:00,94.063528,99.213975,93.593869,84.868628,90.974541,69.156250,0.000000,61.275862,0.0,1.060048
2023-10-17 16:13:20,94.835756,99.067247,95.416897,88.063403,91.330237,64.750000,0.000000,63.275862,0.0,1.038257
2023-10-17 16:13:40,93.567144,99.964944,94.096141,86.123425,92.908906,66.906250,0.000000,64.928571,0.0,1.062370
2023-10-17 16:14:00,92.150706,101.219152,90.821216,83.494384,93.286394,75.032258,19.103448,60.793103,0.0,1.114488
2023-10-17 16:14:20,90.833428,100.648262,88.791894,80.487381,92.247284,86.406250,33.785714,59.500000,0.0,1.133530
...,...,...,...,...,...,...,...,...,...,...
2023-10-13 14:55:40,97.990398,99.141016,89.786491,80.571006,88.462028,67.328125,39.107143,91.714286,0.0,1.104186
2023-10-13 14:56:00,98.738189,99.106112,91.051269,81.310068,92.582797,62.784615,38.928571,91.000000,0.0,1.088465
2023-10-13 14:56:20,97.644569,97.781563,89.892325,79.747017,91.389403,66.734375,18.035714,87.821429,0.0,1.087763
2023-10-13 14:56:40,98.487597,96.751565,89.677660,79.865295,92.926508,60.646154,18.428571,83.928571,0.0,1.078881


In [ ]:
# combined_df.to_csv(r"C:\Users\ballj\OneDrive\바탕 화면\EEG_jm_all.csv")